# Relatório PDS

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
data_path = 'dois_processos_do_alunx_2808.npz'

In [2]:
max_width = 6
params = {
    
    "legend.fontsize": 8,
    "legend.edgecolor": 'black',
    "grid.color": "k",
    "grid.linestyle": ":",
    "grid.linewidth": 0.5,
}
matplotlib.rcParams.update(params)
aspect_ratio = 1.2
figsize = (max_width, max_width / aspect_ratio)

## Primeiramente vamos extrair os dados armazenados

In [3]:
x = np.load(data_path)
X1 = x['X1']
X2 = x['X2']

Cada processo estocastico  é composto por 400 realizações, com cada uma dessas tendo 190 amostras.

In [4]:
X1.shape

(400, 190)

# Questão 1 e 2

## Para Calcular a média de um processo estocastico no tempo temos:
$E(X[n=1]) = \frac{1}{N} \sum^{N}_{i=0}x_i[n=1] $, onde $X$ é o processo estocastico, $x_i[n=1]$ é a variável aleatoria que denota a realização $i$ no instante $n=1$, e por ultimo $N$ é o numero de realizações desse processo estocástico. 

In [5]:
def curva_media(x):
    """ Curva media 
    
    Calcula a curva que representa a média do processo estocastico x e retorna
    
    Args:
        x : Processo estocastico.
    
    Returns:
        mean: Curva da média
    """
    
    mean = np.zeros(x.shape[1])
    for i in range(x.shape[1]):
        mean[i] = np.mean(x[:, i])
        
    return mean

In [6]:
E_x1 = curva_media(X1)
E_x2 = curva_media(X2)

## Assim temos:

In [24]:
def analisis_1_plot(X, E_x, name, ts=0.25, save=True):
    time_array = np.arange(190)*ts
    plt.figure(1, figsize=figsize)
    ax1 = plt.subplot(211)
    ax1.plot(time_array, X[0], linestyle="--",marker=".", label='x_0[n]', markersize=4)
    ax1.plot(time_array, X[1], linestyle="--",marker=".", label='x_1[n]', markersize=4)
    ax1.plot(time_array, X[2], linestyle="--",marker=".", label='x_2[n]', markersize=4)
    ax1.legend(loc='upper center',
                  bbox_to_anchor=(0.7, 0.15),
                  fancybox=False,
                  ncol=3)
    ax1.set_ylabel('Valor')
    ax2 = plt.subplot(212)
    ax2.plot(time_array, E_x, linestyle="--",marker="o", label='Média m(t)')
    ax2.legend()
    plt.xlabel("Tempo(s)")
    plt.ylabel("Média")
    if save:
        fig_name = f"1-{name}.eps"
        plt.savefig(fig_name, format="eps", dpi=300, bbox_inches="tight")
    else:
        plt.show()
    plt.close()


In [25]:
analisis_1_plot(X1, E_x1,'X1')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [26]:
analisis_1_plot(X2, E_x2, 'X2')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# Questão 2

In [29]:
def fdp(instante1, instante2, X, name, save=True):
    n_bins = 50
    plt.figure(1, figsize=figsize)
    ax1 = plt.subplot(211)
    ax1.hist(X[:,instante1], bins=n_bins, density=True, label=f'f.d.p({name}_45[n])', color = 'blue', edgecolor = 'black')
    ax1.set_ylabel("Probabilidade")
    ax1.legend()
    ax2 = plt.subplot(212)
    ax2.hist(X[:,instante2], bins=n_bins, density=True, label=f'f.d.p({name}_120[n])', color = 'red', edgecolor = 'black')
    ax2.set_xlabel("Valor")
    ax2.set_ylabel("Probabilidade")
    ax2.legend()
    if save:
        fig_name = f"2_{name}.eps"
        plt.savefig(fig_name, format="eps", dpi=300, bbox_inches="tight")
    else:
        plt.show()
    plt.close()

In [30]:
fdp(45, 120, X1,'X1')
fdp(45, 120, X2,'X2')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


#  Questão 4

In [12]:
def estimate_auto_correlation(X, instant1, instant2):
    return np.mean(X[:,instant1]*X[:,instant2])

In [13]:
rx1 = estimate_auto_correlation(X1, 45, 120)
rx2 = estimate_auto_correlation(X1, 55, 130)
rx3 = estimate_auto_correlation(X1, 65, 140)
print(f"rx1: {rx1}\nrx2:{rx2}\nrx3: {rx3}")

rx1: 0.07238173826116848
rx2:-0.025789722072666627
rx3: -0.2558903986121387


In [14]:
def autocorrelation_in_n(X, k):
    rx_n = np.zeros(X.shape[1]-k)
    for i in range(X.shape[1]-k):
        instant1 = i
        instant2 = instant1 + k
        rx_n[i] = estimate_auto_correlation(X, instant1, instant2)
    return rx_n

In [15]:
def autocorrelation_plot(rx, k, name, ts=0.5, save=True, n_steps=190):
    plt.figure(1, figsize=(5,6))
    axes = {}
    markers = ['^', 'v', 'd']
    colors = ['b', 'r', 'y']
    
    for sig, k, i in zip(rx, k, range(len(rx))):
        time_array = (np.arange(sig.shape[0]) * ts)+ (n_steps-k)*ts
        plot_n = int(100*len(rx)+10+i+1)
        axes[f'{i}'] = plt.subplot(plot_n)
        axes[f'{i}'].plot(time_array, sig, linestyle="--",marker=markers[i],
                          markersize=4,
                          color=colors[i],
                          label=f'cov{i+1}, k={k}')
        axes[f'{i}'].legend()
        axes[f'{i}'].set_xlabel('Tempo(s)')
        axes[f'{i}'].set_ylabel('Valor')
    if save:
        fig_name = f"3_autocorr_{name}.eps"
        plt.savefig(fig_name, format="eps", dpi=300, bbox_inches="tight")
    else:
        plt.show()
    plt.close()


In [16]:
k_i = [20,35,54]
rx_x1 = [0,0,0]
rx_x2 = [0,0,0]
for k, i in zip(k_i, range(len(k_i))):
    rx_x1[i] = autocorrelation_in_n(X1,k)
    rx_x2[i] = autocorrelation_in_n(X2,k)

autocorrelation_plot(rx_x1, k_i,'X1',ts=0.5, save=True)
autocorrelation_plot(rx_x2, k_i,'X2',ts=0.5, save=True)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

# Questão 5

In [17]:
def estimate_autocovariance(X, instant1, instant2):
    return np.mean((X[:,instant1]-np.mean(X[:,instant1]))*(X[:,instant2]-np.mean(X[:,instant2])))

In [18]:
cov1 = estimate_autocovariance(X1, 45, 120)
cov2 = estimate_autocovariance(X1, 55, 130)
cov3 = estimate_autocovariance(X1, 65, 140)
print(f"cov1: {cov1}\ncov2:{cov2}\ncov3: {cov3}")

cov1: 0.06468325449284917
cov2:-0.03207957539230847
cov3: -0.2447286621839256


In [19]:
def autocovariance_in_n(X, k):
    sigx_n = np.zeros(X.shape[1]-k)
    for i in range(X.shape[1]-k):
        instant1 = i
        instant2 = instant1 + k
        sigx_n[i] = estimate_autocovariance(X, instant1, instant2)
    return sigx_n

In [20]:
def autocovariance_plot(sigx, k, name, ts=0.5, save=True, n_steps=190):
    plt.figure(1, figsize=(5,6))
    axes = {}
    markers = ['^', 'v', 'd']
    colors = ['b', 'r', 'y']
    
    for sig, k, i in zip(sigx, k_i, range(len(sigx))):
        time_array = (np.arange(sig.shape[0]) * ts)+ (n_steps-k)*ts
        plot_n = int(100*len(sigx)+10+i+1)
        axes[f'{i}'] = plt.subplot(plot_n)
        axes[f'{i}'].plot(time_array, sig, linestyle="--",marker=markers[i],
                          markersize=4,
                          color=colors[i],
                          label=f'cov{i+1}, k={k}')
        axes[f'{i}'].legend()
        axes[f'{i}'].set_xlabel('Tempo(s)')
        axes[f'{i}'].set_ylabel('Valor')
    if save:
        fig_name = f"4_autocov_{name}.eps"
        plt.savefig(fig_name, format="eps", dpi=300, bbox_inches="tight")
    else:
        plt.show()
    plt.close()


In [21]:
k_i = [20,35,54]
sig_x1= [0,0,0]
sig_x2= [0,0,0]
for k, i in zip(k_i, range(len(k_i))):
    sig_x1[i] = autocovariance_in_n(X1,k)
    sig_x2[i] = autocovariance_in_n(X2,k)
    
autocovariance_plot(sig_x1, k_i, 'X1', ts=0.5, save=True)
autocovariance_plot(sig_x1, k_i, 'X2', ts=0.5, save=True)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back